In [ ]:
import debugpy
debugpy.configure(subProcess=False)

In [ ]:
import os
import sys
from pathlib import Path
# Add parent directory to path (most common use case)
cwd = Path(os.getcwd())
if cwd.name == 'notebooks':
    sys.path.append(os.path.abspath('..'))

In [ ]:
if cwd.name == 'notebooks':
    os.chdir('..')

In [ ]:
import math
from dotenv import load_dotenv

from typing import Tuple
from itertools import zip_longest

import orjson
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import expon, weibull_min, lognorm
from scipy.optimize import minimize

import warnings

# Visualisation
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import seaborn as sns



In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Load environment variables
load_dotenv() 
# Remove warnings
warnings.filterwarnings("ignore") 


# Trying to determine optimal timing to look back per correlation type

Currently, when identifying correlations with a water anomaly, Marvin uses a fixed time window of 1h for poultry and 2h for pigs and tries to identify factors that are possibly correlated with the anomaly. There is now the ability to determine individual look-back windows per type of correlation. This notebook tries to assist in the determination of the optimal lookup-back time for Marvin per correlation (type).

**Dataset:**  
The dataset we use for the analysis has anomalies (per farm and shed) and their correlations. That is for each anomaly, we have as many entries as correlating factors Marvin has identified in the lookback window. 

**Hypothesis:**  
Assuming that the correlating factor "causes" the anomaly, it stands to reason to assume that of the correlating factor is "present" during a time, a number of anomalies will be caused with this specific correlating factor. 

**Sub-hypothesis 1:**
If we measure the time difference between an anomaly and all possible correlation factors, we should how frequent another anomaly with that correlating factor occurs in the past.

**Sub-hypothesis 2:**
If we measure the time difference for all correlating factors of an anomaly, we would see how far back the same correlating factor occurs in other anomalies.

**Limitations:**
- The animal type is not provided

In [ ]:
PROCESS_BY_CATEGORY = True

In [ ]:
SRCDIR = Path("/mnt/c/Users/heine/Downloads/rainfall")

In [ ]:
files  = [f for f in SRCDIR.iterdir() if f.suffix == '.json']
df = pd.DataFrame() 
for f in files:
    with open(f, 'r') as infile:
        data = orjson.loads(infile.read())
        t_df = pd.json_normalize(data)
        t_df['LocalTime'] = pd.to_datetime(t_df['LocalTime']) # type:ignore
        t_df = t_df[['AnomalyId', 'LocalTime', 'FarmId', 'FarmName', 'ShedId', 'ShedName','Correlation','Category']]
        df = pd.concat([df, t_df], ignore_index=True)
df = df.sort_values(by='LocalTime')
df = df.reset_index(drop=True)

In [ ]:
df['Correlation'] = [corr.strip() for corr in df['Correlation']]
correlations = df['Correlation'].sort_values().unique()
categories = df['Category'].unique()
logger.info(f"Found {len(correlations)} unique correlations and {len(categories)} unique categories.")

In [ ]:
def order_correlations_by_pairs(correlations: list|np.ndarray) -> Tuple[int, list]:

    pairs = []
    singles = []

    special_pairs = [('Sunset','Sunrise'),('Lights On', 'Lights Off')]

    for correlation in correlations:
        if correlation.endswith('Increased'):
            cat_name = correlation.split('Increased')[0]
            if f"{cat_name}Decreased" in correlations:
                pairs.append(correlation)
        elif correlation.endswith('Decreased'):
            cat_name = correlation.split('Decreased')[0]
            if f"{cat_name}Increased" in correlations:
                pairs.append(correlation)
        elif any([correlation in pair for pair in special_pairs]):
            for pair in special_pairs:
                if correlation in pair:
                    other = pair[1] if correlation == pair[0] else pair[0]
                    break
            if other in correlations:
                pairs.append(correlation)
            else:
                singles.append(correlation)
        else:
            singles.append(correlation)
    
    pairs.sort() # automatic ordering
    singles.sort()

    return len(pairs+singles), pairs + singles

def order_correlations_by_category(correlations: list|np.ndarray, df:pd.DataFrame) -> Tuple[int, list]:

    pairs = []
    singles = []
    categories = df['Category'].unique()
    assert len(categories)==2

    for correlation in correlations:
            if len(df[df['Correlation']==correlation]['Category'].unique()) == 2:
                 pairs.append(correlation)
            else:
                singles.append(correlation)
    
    pairs.sort() # automatic ordering
    singles.sort()

    return len(pairs)*2+len(singles),pairs + singles


if PROCESS_BY_CATEGORY:
    number_of_plots, correlations_ordered = order_correlations_by_category(correlations=correlations, df=df)
else:
    number_of_plots, correlations_ordered = order_correlations_by_pairs(correlations=correlations)

In [ ]:
farmids = df['FarmId'].unique()
shedids = df['ShedId'].unique()
logger.info(f"Found {len(farmids)} farms {len(shedids)} sheds.")

In [ ]:
start_of_dataset = df['LocalTime'].min()
max_lookback_length = 4 # hours
earliest_time = start_of_dataset + pd.Timedelta(hours=max_lookback_length)

In [ ]:
anomalyids = df['AnomalyId'].unique()
logger.info(f"Found {len(anomalyids)} unique anomalies")

In [ ]:
# Find all unique anomalies for each shed at least 2h after earliest_time
anomalies = df.groupby(by=['ShedId', 'AnomalyId', 'Category']).agg({'LocalTime':'min'}).reset_index()
anomalies = anomalies[anomalies['LocalTime'] > earliest_time]
logger.info(f"Found {len(anomalies)} unique anomalies after {earliest_time}.")


In [ ]:
anomalies

In [ ]:
# Now categorize delays into 15-min intervals
interval_labels = [f"{i*15}-{(i+1)*15}min" for i in range(int(max_lookback_length * 60 / 15))]

## Utility functions

In [ ]:
def fit_times(x: np.ndarray) -> Tuple[str,dict]:

    x = np.sort(x)

    shape, loc, scale = stats.lognorm.fit(x, floc=0)

    # Fit Weibull using scipy's weibull_min
    c, loc_w, scale_w = stats.weibull_min.fit(x, floc=0)

    # Fit exponential
    loc_e, scale_e = stats.expon.fit(x, floc=0)

    # Compute log-likelihoods
    ll_logn  = np.sum(stats.lognorm.logpdf(x, shape, loc, scale))
    ll_weib  = np.sum(stats.weibull_min.logpdf(x, c, loc_w, scale_w))
    ll_exp   = np.sum(stats.expon.logpdf(x, loc_e, scale_e))

    ret = {
        'logn' : 
            { 
                'log-likelihood': ll_logn,
                'shape': shape,
                'loc': loc,
                'scale': scale
            },
        'weib':
            { 
                'log-likelihood': ll_weib,
                'c': c,
                'loc_w': loc_w,
                'scale_w': scale_w
            },
        'exp'  : 
                { 
                'log-likelihood': ll_exp,
                'loc_e': loc_e,
                'scale_e': scale_e
            }
    }
    maxvalue = max(ll_logn, ll_weib, ll_exp)
    maxtype = 'logn'
    for type,value in ret.items():
        if value['log-likelihood'] == maxvalue:
            maxtype = type
            break

    return maxtype, ret

def determine_type(correlations: list, corr_data: pd.DataFrame) -> pd.DataFrame:
    ret = pd.DataFrame()
    for i,correlation in enumerate(correlations):
        delays = corr_data[corr_data['Correlation']==correlation]['Delay']
        type, value_dict = fit_times(delays)
        parameters = ['p1', 'p2', 'p3']
        values = value_dict[type]
        log_likelihood = values.pop('log-likelihood')
        ps = dict(zip_longest(parameters, list(values.values()), fillvalue=None))

        t_df = pd.DataFrame({
            'Correlation': correlation,
            'Type': type,
            'Log-Likelihood': log_likelihood,
            **ps
        }, index=[i])
        ret = pd.concat([ret, t_df], ignore_index=True)
    return ret


In [ ]:
"""
Ultra-simple mixture distribution fitting
Minimal dependencies, straightforward code
"""



def fit_mixture_simple(data, dist1='expon', dist2='weibull_min'):
    """
    Fit a two-component mixture distribution
    
    Parameters
    ----------
    data : array
        Your observations (e.g., time delays)
    dist1 : str
        Component 1: 'expon', 'weibull_min', or 'lognorm'
    dist2 : str
        Component 2: 'expon', 'weibull_min', or 'lognorm'
    
    Returns
    -------
    dict with fitted parameters and statistics
    """
    
    data = np.asarray(data, dtype=float)
    data = data[~np.isnan(data)]  # Remove NaN values
    
    dists = {
        'expon': expon,
        'weibull_min': weibull_min,
        'lognorm': lognorm,
    }
    
    D1 = dists[dist1]
    D2 = dists[dist2]
    
    # Fit individual distributions for starting guesses
    p1 = D1.fit(data)
    p2 = D2.fit(data)
    
    # Objective function: negative log-likelihood
    def objective(params):
        lam = params[0]
        
        # Ensure lambda is valid
        if lam <= 0 or lam >= 1:
            return 1e10
        
        # Extract parameters for each distribution
        p1_est = params[1:len(p1)+1]
        p2_est = params[len(p1)+1:]
        
        try:
            # Evaluate mixture density
            pdf1 = D1.pdf(data, *p1_est)
            pdf2 = D2.pdf(data, *p2_est)
            
            # Clamp to avoid log(0)
            mix = np.clip(lam * pdf1 + (1-lam) * pdf2, 1e-10, None)
            
            return -np.sum(np.log(mix))
        except:
            return 1e10
    
    # Initial guess
    x0 = [0.5] + list(p1) + list(p2)
    
    # Optimize
    res = minimize(objective, x0, method='Nelder-Mead',
                   options={'maxiter': 5000})
    
    # Extract results
    lam = np.clip(res.x[0], 0.01, 0.99)
    p1_fit = res.x[1:len(p1)+1]
    p2_fit = res.x[len(p1)+1:]
    
    # Calculate statistics
    n_params = len(res.x)
    nll = res.fun
    aic = 2*nll + 2*n_params
    bic = 2*nll + n_params*np.log(len(data))
    
    return {
        'lambda': lam,
        'params1': p1_fit,
        'params2': p2_fit,
        'dist1': dist1,
        'dist2': dist2,
        'nll': nll,
        'aic': aic,
        'bic': bic,
        'n': len(data),
    }


def plot_mixture(data, result, bins=40):
    """Plot data histogram with fitted mixture overlay"""
    
    D1 = {'expon': expon, 'weibull_min': weibull_min, 'lognorm': lognorm}[result['dist1']]
    D2 = {'expon': expon, 'weibull_min': weibull_min, 'lognorm': lognorm}[result['dist2']]
    
    fig, ax = plt.subplots(figsize=(12, 5))
    
    # Histogram
    ax.hist(data, bins=bins, density=True, alpha=0.5, color='gray', edgecolor='black')
    
    # Fitted curves
    x = np.linspace(np.percentile(data, 0.5), np.percentile(data, 99.5), 300)
    
    pdf1 = D1.pdf(x, *result['params1'])
    pdf2 = D2.pdf(x, *result['params2'])
    
    w1 = result['lambda']
    w2 = 1 - result['lambda']
    
    ax.plot(x, w1*pdf1, 'r-', lw=2.5, label=f'{result["dist1"]} ({w1:.1%})')
    ax.plot(x, w2*pdf2, 'b-', lw=2.5, label=f'{result["dist2"]} ({w2:.1%})')
    ax.plot(x, w1*pdf1 + w2*pdf2, 'k--', lw=3, label='Mixture')
    
    ax.set_xlabel('Time Delay')
    ax.set_ylabel('Density')
    ax.set_title(f'{w1:.1%} {result["dist1"]} + {w2:.1%} {result["dist2"]} (BIC={result["bic"]:.1f})')
    ax.legend()
    ax.grid(alpha=0.3)
    
    return fig


def print_results(result):
    """Print results nicely"""
    logger.info("\n" + "="*60)
    logger.info(f"MIXTURE: {result['lambda']:.1%} {result['dist1']} + {1-result['lambda']:.1%} {result['dist2']}")
    logger.info("="*60)
    logger.info(f"λ (mixture weight):     {result['lambda']:.6f}")
    logger.info(f"params1 ({result['dist1']}):        {result['params1']}")
    logger.info(f"params2 ({result['dist2']}):        {result['params2']}")
    logger.info(f"Negative LL:            {result['nll']:.2f}")
    logger.info(f"AIC:                    {result['aic']:.2f}")
    logger.info(f"BIC:                    {result['bic']:.2f}")
    logger.info(f"N samples:              {result['n']}")
    logger.info("="*60 + "\n")


In [ ]:

def determine_type_by_category(correlations: list, corr_data: pd.DataFrame) -> pd.DataFrame:
    ret = pd.DataFrame()
    categories = corr_data['Category'].unique()
    assert len(categories) == 2
    for i,correlation in enumerate(correlations):
        for category in categories:
            delays = corr_data[(corr_data['Correlation']==correlation) & (corr_data['Category']==category)]['Delay']
            if delays.empty:
                type = 'No data'
                log_likelihood = None
                ps = {'p1': None, 'p2': None, 'p3': None}
            else:
                type, value_dict = fit_times(delays)
                parameters = ['p1', 'p2', 'p3']
                values = value_dict[type]
                log_likelihood = values.pop('log-likelihood')
                ps = dict(zip_longest(parameters, list(values.values()), fillvalue=None))

            t_df = pd.DataFrame({
                'Correlation': correlation,
                'Category': category,
                'Type': type,
                'Log-Likelihood': log_likelihood,
                **ps
            }, index=[i])
            ret = pd.concat([ret, t_df], ignore_index=True)
    return ret

In [ ]:
def find_best_fit(delays:pd.Series):
    best_bic = float('inf')
    best_result = None
    models = [
        ('expon', 'weibull_min'),
        ('expon', 'lognorm'),
        ('weibull_min', 'lognorm'),
    ]
    for dist1, dist2 in models:
        result = fit_mixture_simple(delays, dist1=dist1, dist2=dist2)
        if result['bic'] < best_bic:
            best_bic = result['bic']
            best_result = result
    return best_result

In [ ]:

delays = merged[(merged['Correlation']=='CO2 Decreased') & (merged['Category']=='Decreased Water')]['Delay']
result = find_best_fit(delays)

In [ ]:
fig = plot_mixture(delays, result)

In [ ]:
def determine_type_by_category_mixed(correlations: list, corr_data: pd.DataFrame) -> pd.DataFrame:
    ret = pd.DataFrame()
    categories = corr_data['Category'].unique()
    assert len(categories) == 2
    for i,correlation in enumerate(correlations):
        for category in categories:
            delays = corr_data[(corr_data['Correlation']==correlation) & (corr_data['Category']==category)]['Delay']
            if delays.empty:
                type = 'No data'
                log_likelihood = None
                ps = {'p1': None, 'p2': None, 'p3': None}
            else:
                type, value_dict = fit_times(delays)
                parameters = ['p1', 'p2', 'p3']
                values = value_dict[type]
                log_likelihood = values.pop('log-likelihood')
                ps = dict(zip_longest(parameters, list(values.values()), fillvalue=None))

            t_df = pd.DataFrame({
                'Correlation': correlation,
                'Category': category,
                'Type': type,
                'Log-Likelihood': log_likelihood,
                **ps
            }, index=[i])
            ret = pd.concat([ret, t_df], ignore_index=True)
    return ret

In [ ]:
def plot_subset(ax: Axes, subset: pd.DataFrame, n_sample: int, corr: str, 
                types:pd.DataFrame|None=None, category:str|None=None, cumulative: bool=True) -> None:
    if subset.empty:
        ax.set_title(f'Correlation: {corr} {category} (No Data)')
        ax.axis('off')  # Remove empty plot
        return
    # Calculate percentage and cumulative percentage
    subset['Percent'] = 100 * subset['Count'] / subset['Count'].sum()
    subset = subset.sort_values('DelayInterval')
    subset['CumPercent'] = subset['Percent'].cumsum()
    
    # Assign colors: blue for first 90%, orange for rest
    colors = ['#1f77b4' if cum <= 90 else '#ff7f0e' for cum in subset['CumPercent']]
    
    sns.barplot(data=subset, x='DelayInterval', y='Percent', ax=ax, palette=colors)
    
    if cumulative:
        # Add cumulative line on secondary axis
        ax2 = ax.twinx()
        ax2.plot(range(len(subset)), subset['CumPercent'].values, color='darkred', linestyle='-', linewidth=2, label='Cumulative %') # type:ignore
        ax2.axhline(y=90, color='darkred', linestyle='--', alpha=0.5, label='90% threshold')
        ax2.set_ylabel('Cumulative %', color='darkred')
        ax2.tick_params(axis='y', labelcolor='darkred')
        ax2.set_ylim(0, 105)

    # Plot distribution fit if available
    if isinstance(types, pd.DataFrame) and not types.empty:
        if 'Category' in types.columns:
            type = types[(types['Correlation'] == corr) & (types['Category'] == category)].iloc[0]
        else:
            type = types[types['Correlation'] == corr].iloc[0]
        match type['Type']:
            case 'weib':
                c, loc, scale = type['p1'], type['p2'], type['p3']
                p1 = f"{c:.2f}"
                p2 = f"{scale:.2f}"
                dist = stats.weibull_min(c, loc, scale)
            case 'logn':
                shape, loc, scale = type['p1'], type['p2'], type['p3']
                p1 = f"{shape:.2f}"
                p2 = f"{scale:.2f}"
                dist = stats.lognorm(shape, loc, scale)
            case 'exp':
                loc, scale = type['p1'], type['p2']
                p1 = ''
                p2 = f"{scale:.2f}"
                dist = stats.expon(loc, scale)
            case _:
                p1, p2 = '', ''
                dist = None
        
        if dist:
            # Integrate PDF over each 15-min interval to get expected percentage
            interval_percentages = []
            for interval_start in range(0, max_lookback_length*60, 15):
                interval_end = interval_start + 15
                # CDF gives cumulative probability, so CDF(b) - CDF(a) gives probability in [a,b]
                prob = dist.cdf(interval_end) - dist.cdf(interval_start)
                percentage = prob * 100
                interval_percentages.append(percentage)
            
            # Plot at categorical x positions (matching the bars)
            legend_label = f"{type['Type']}\nshape: {p1}\nscale: {p2}"
            ax.plot(range(len(interval_percentages)), interval_percentages, 'r-', linewidth=2, alpha=0.7, label=legend_label, marker='o')
        else:
            legend_label = "Unknown distribution"

    title = f'{corr}'
    if category is not None:
        title += f', {category}'
    if isinstance(types, pd.DataFrame) and not types.empty:
        title += f' N={n_sample:,}'
    ax.set_title(f'{corr} {category} N={n_sample:,}')
    ax.set_xlabel('Delay Interval')
    ax.set_ylabel('Percent')
    ax.tick_params(axis='x', rotation=45)
    if isinstance(types, pd.DataFrame) and not types.empty:
        ax.legend()

def plot(number_of_plots:int, result: pd.DataFrame, df: pd.DataFrame, correlations:list,
         types:pd.DataFrame|None=None, category:str|None=None, cumulative: bool=True) -> None:
    rows = number_of_plots + 1
    rows = math.ceil(rows/2)
    columns = 2
    fig, axs = plt.subplots(rows, columns, figsize=(20,rows*4+20))

    # Add overall figure title
    fig.suptitle(f'Correlation Delay Analysis\n{len(anomalyids):,} anomalies from {len(farmids):,} farms and {len(shedids):,} sheds', 
                fontsize=16, fontweight='bold', y=1.01)

    axs = axs.flatten() if rows > 1 else [axs]
    j=0
    for corr in correlations:
        if PROCESS_BY_CATEGORY:
            subset = result[(result['Correlation'] == corr) & (result['Category'] == categories[0])].copy()
            n_sample = len(df[(df['Correlation']==corr) & (df['Category'] == categories[0])])
            plot_subset(ax=axs[j], subset=subset, n_sample=n_sample, corr=corr, types=types, category=categories[0], cumulative=cumulative)
            j += 1
            subset = result[(result['Correlation'] == corr) & (result['Category'] == categories[1])].copy()
            n_sample = len(df[(df['Correlation']==corr) & (df['Category'] == categories[1])])
            plot_subset(ax=axs[j], subset=subset, n_sample=n_sample, corr=corr, types=types, category=categories[1], cumulative=cumulative)
            j += 1
        else:
            n_sample = len(df[df['Correlation']==corr])
            subset = result[result['Correlation'] == corr].copy()
            plot_subset(ax=axs[j], subset=subset, n_sample=n_sample, corr=corr, types=types, cumulative=cumulative)
            j += 1
    
    # Remove any remaining empty subplots
    for k in range(j, len(axs)):
        axs[k].axis('off')

    plt.tight_layout()
    plt.show()

## Sub-hypothesis 1:
Measuring the time difference between an anomaly and all possible correlation factors, we should how frequent another anomaly with that correlating factor occurs in the past.

In [ ]:
RUN_HYPOTHESIS_1 = True

In [ ]:
if RUN_HYPOTHESIS_1:
    if not PROCESS_BY_CATEGORY:
        # Self-join approach
        merged = anomalies.merge(df, on='ShedId', suffixes=('_anomaly', '_corr'))

        # Filter: correlation must be before anomaly
        merged = merged[merged['LocalTime_corr'] < merged['LocalTime_anomaly']]

        # Calculate time difference
        merged['Delay'] = (merged['LocalTime_anomaly'] - merged['LocalTime_corr']).dt.total_seconds() / 60

        # Filter: only within lookback window
        merged = merged[merged['Delay'] <= max_lookback_length * 60]

        # Categorize and count
        merged['DelayInterval'] = pd.cut(merged['Delay'], bins=range(0, max_lookback_length*60+1, 15), labels=interval_labels, right=False)
        result = merged.groupby(['Correlation', 'DelayInterval']).size().reset_index(name='Count')
    else:
        # Process each category separately
        results = []
        mergeds = []
        for category in categories:
            anomalies_cat = anomalies[anomalies['Category'] == category]
            df_cat = df[df['Category'] == category]

            merged = anomalies_cat.merge(df_cat, on=['ShedId','Category'], suffixes=('_anomaly', '_corr'))

            # Filter: correlation must be before anomaly
            merged = merged[merged['LocalTime_corr'] < merged['LocalTime_anomaly']]


            # Calculate time difference
            merged['Delay'] = (merged['LocalTime_anomaly'] - merged['LocalTime_corr']).dt.total_seconds() / 60

            # Filter: only within lookback window
            merged = merged[merged['Delay'] <= max_lookback_length * 60]

            # Categorize and count
            merged['DelayInterval'] = pd.cut(merged['Delay'], bins=range(0, max_lookback_length*60+1, 15), labels=interval_labels, right=False)
            mergeds.append(merged)
            result_cat = merged.groupby(['Correlation', 'Category', 'DelayInterval']).size().reset_index(name='Count')
            results.append(result_cat)
        
        merged = pd.concat(mergeds, ignore_index=True)
        result = pd.concat(results, ignore_index=True)

In [ ]:
if RUN_HYPOTHESIS_1:
    if PROCESS_BY_CATEGORY:
        types = determine_type_by_category(correlations=correlations_ordered, corr_data=merged)
    else:
        types = determine_type(correlations=correlations_ordered, corr_data=merged)

In [ ]:
if RUN_HYPOTHESIS_1:
    # Alternative: with cumulative line
    plot(number_of_plots=number_of_plots, result=result, df=df, correlations=correlations_ordered, types=types, cumulative=False)


## Sub-hypothesis 2:
If we measure the time difference for all correlating factors of an anomaly, we would see how far back the same correlating factor occurs in other anomalies.

In [ ]:
RUN_HYPOTHESIS_2 = True

In [ ]:
def get_timedifferences(df: pd.DataFrame, anomalies: pd.DataFrame, max_lookback_length: int) -> pd.DataFrame:
    # Different approach: for each anomaly, only look at previous occurrences of its associated correlations
    # For each anomaly-correlation pair, find previous occurrences of that SAME correlation type

    # Start with the full dataset (each row is an anomaly-correlation pair)
    result_list = []

    for _, anomaly_row in anomalies.iterrows():
        shed_id = anomaly_row['ShedId']
        anomaly_id = anomaly_row['AnomalyId']
        anomaly_time = anomaly_row['LocalTime']
        
        # Get all correlations associated with this specific anomaly
        anomaly_correlations = df[(df['ShedId'] == shed_id) & (df['AnomalyId'] == anomaly_id)]['Correlation'].unique()
        
        # For each correlation type associated with this anomaly
        for corr_type in anomaly_correlations:
            # Find previous occurrences of this SAME correlation type in the same shed
            previous_same_corr = df[
                (df['ShedId'] == shed_id) & 
                (df['Correlation'] == corr_type) & 
                (df['LocalTime'] < anomaly_time)
            ]
            
            # Calculate time differences
            for _, prev_row in previous_same_corr.iterrows():
                delay_minutes = (anomaly_time - prev_row['LocalTime']).total_seconds() / 60
                
                # Only include if within lookback window
                if delay_minutes <= max_lookback_length * 60:
                    result_list.append({
                        'AnomalyId': anomaly_id,
                        'ShedId': shed_id,
                        'Correlation': corr_type,
                        'Delay': delay_minutes
                    })

    return  pd.DataFrame(result_list)

def get_timedifferences_per_category(df: pd.DataFrame, anomalies: pd.DataFrame, max_lookback_length: int) -> pd.DataFrame:
    # Different approach: for each anomaly, only look at previous occurrences of its associated correlations
    # For each anomaly-correlation pair, find previous occurrences of that SAME correlation type

    # Start with the full dataset (each row is an anomaly-correlation pair)
    result_list = []
    categories = anomalies['Category'].unique()
    assert len(categories) == 2

    for _, anomaly_row in anomalies.iterrows():
        shed_id = anomaly_row['ShedId']
        anomaly_id = anomaly_row['AnomalyId']
        anomaly_time = anomaly_row['LocalTime']
        category = anomaly_row['Category']
        
        # Get all correlations associated with this specific anomaly
        anomaly_correlations = df[(df['ShedId'] == shed_id) & (df['AnomalyId'] == anomaly_id) & (df['Category'] == category)]['Correlation'].unique()
        
        # For each correlation type associated with this anomaly
        for corr_type in anomaly_correlations:
            # Find previous occurrences of this SAME correlation type in the same shed
            previous_same_corr = df[
                (df['ShedId'] == shed_id) & 
                (df['Correlation'] == corr_type) & 
                (df['Category'] == category) &
                (df['LocalTime'] < anomaly_time)
            ]
            
            # Calculate time differences
            for _, prev_row in previous_same_corr.iterrows():
                delay_minutes = (anomaly_time - prev_row['LocalTime']).total_seconds() / 60
                
                # Only include if within lookback window
                if delay_minutes <= max_lookback_length * 60:
                    result_list.append({
                        'AnomalyId': anomaly_id,
                        'ShedId': shed_id,
                        'Correlation': corr_type,
                        'Category': category,
                        'Delay': delay_minutes
                    })

    return pd.DataFrame(result_list)


In [ ]:
if RUN_HYPOTHESIS_2:
    if PROCESS_BY_CATEGORY:
        result2 = get_timedifferences_per_category(df, anomalies, max_lookback_length)
            # Categorize and count
        result2['DelayInterval'] = pd.cut(result2['Delay'], bins=range(0, max_lookback_length*60+1, 15), labels=interval_labels, right=False)
        merged2 = result2.groupby(['Correlation', 'Category','DelayInterval']).size().reset_index(name='Count')
    else:
        result2 = get_timedifferences(df, anomalies, max_lookback_length)
        # Categorize and count
        result2['DelayInterval'] = pd.cut(result2['Delay'], bins=range(0, max_lookback_length*60+1, 15), labels=interval_labels, right=False)
        merged2 = result2.groupby(['Correlation', 'DelayInterval']).size().reset_index(name='Count')
    
    logger.info(f"Sub-hypothesis 2: Found {len(result2)} correlation occurrences")

In [ ]:
if RUN_HYPOTHESIS_2:
    if PROCESS_BY_CATEGORY:
        types = determine_type_by_category(correlations=correlations_ordered, corr_data=result2)
    else:
        types = determine_type(correlations=correlations_ordered, corr_data=result2)

In [ ]:
if RUN_HYPOTHESIS_2:
    # Alternative: with cumulative line
    plot(number_of_plots=number_of_plots, result=merged2, df=df, correlations=correlations_ordered, types=types, cumulative=False)